In [13]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time

# Configurar o driver do Selenium
options = webdriver.ChromeOptions()
# options.add_argument('--headless')  # Remover o comentário para rodar o navegador em modo invisível
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Função para extrair latitude e longitude da URL do mapa
def extract_lat_long(map_url):
    try:
        # Localiza o índice onde começam as coordenadas, após 'markers='
        start = map_url.find("markers=") + len("markers=")
        end = map_url.find("&", start)  # Encontra o fim do parâmetro 'markers'
        
        # Extrai a string com a latitude e longitude e faz a substituição, se necessário
        lat_long = map_url[start:end].replace("%2C", ",")
        
        # Verifica se lat_long contém duas partes (latitude e longitude)
        parts = lat_long.split(",")
        if len(parts) == 2:
            latitude, longitude = parts
            return latitude, longitude
        else:
            print("Formato de coordenadas inválido:", lat_long)
            return None, None
    except Exception as e:
        print(f"Erro ao extrair latitude/longitude: {e}")
        return None, None
    
# Função para raspar dados de uma URL
def scrape_data(url):
    driver.get(url)
    time.sleep(3)  # Aguarda o carregamento da página

    try:
        # Raspar os preços (aluguel, condomínio, IPTU, seguro incêndio, taxa de serviço, total)
        aluguel = driver.find_element(By.XPATH, "//span[text()='Aluguel']/following::p").text
        condominio = driver.find_element(By.XPATH, "//span[text()='Condomínio']/following::p").text
        iptu = driver.find_element(By.XPATH, "//span[text()='IPTU']/following::p").text
        seguro_incendio = driver.find_element(By.XPATH, "//span[text()='Seguro incêndio']/following::p").text
        taxa_servico = driver.find_element(By.XPATH, "//span[text()='Taxa de serviço']/following::p").text
        total = driver.find_element(By.XPATH, "//span[text()='Total']/following::h4").text

        # Raspar o bairro
        bairro = driver.find_element(By.XPATH, "//small[contains(@class, 'pwAPLE')]").text

        # Raspar informações adicionais (metragem, quartos, banheiros, vagas, andar, aceita pet, sem mobília)
        metragem = driver.find_element(By.XPATH, "//p[contains(text(),'m²')]").text
        quartos = driver.find_element(By.XPATH, "//p[contains(text(),'quarto')]").text
        banheiros = driver.find_element(By.XPATH, "//p[contains(text(),'banheiro')]").text
        vagas = driver.find_element(By.XPATH, "//p[contains(text(),'vaga')]").text
        andar = driver.find_element(By.XPATH, "//p[contains(text(),'andar')]").text
        aceita_pet = driver.find_element(By.XPATH, "//p[contains(text(),'Aceita pet')]").text
        sem_mobilia = driver.find_element(By.XPATH, "//p[contains(text(),'Sem mobília')]").text

        latitude, longitude = None, None  # Inicializa as variáveis para latitude e longitude

        try:
            # Clicar no botão para abrir o mapa, se disponível
            map_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Abrir mapa']"))
            )
            map_button.click()
            time.sleep(5)  # Aguarda o mapa carregar

            # Capturar o `src` da tag <img> que contém o mapa
            try:
                images = driver.find_elements(By.TAG_NAME, 'img')
                for img in images:
                    src = img.get_attribute('src')
                    if src and 'maps.googleapis.com' in src:  # Verifica se src não é None e contém o Google Maps
                        map_url = src
                        latitude, longitude = extract_lat_long(map_url)
                        break  # Para o loop se encontrar a imagem do mapa
                if latitude is None or longitude is None:
                    print("Imagem do mapa não encontrada ou coordenadas inválidas.")
            except Exception as e:
                print(f"Erro ao capturar URL da imagem do mapa: {e}")

            # Verificar se as coordenadas estão presentes no código-fonte da página
            if latitude is None or longitude is None:
                try:
                    page_source = driver.page_source
                    # Procura a linha que contém as coordenadas no HTML (ajustar conforme necessário)
                    start = page_source.find("center=") + len("center=")
                    end = page_source.find("&", start)
                    lat_long = page_source[start:end].replace("%2C", ",")
                    
                    parts = lat_long.split(",")
                    if len(parts) == 2:
                        latitude, longitude = parts
                    else:
                        print("Formato de coordenadas inválido no HTML:", lat_long)
                        latitude, longitude = None, None
                except Exception as e:
                    print(f"Erro ao encontrar coordenadas no HTML: {e}")
        
        except Exception as e:
            print(f"Erro ao interagir com o mapa: {e}")

        # Retornar todas as informações
        return {
            'aluguel': aluguel,
            'condominio': condominio,
            'iptu': iptu,
            'seguro_incendio': seguro_incendio,
            'taxa_servico': taxa_servico,
            'total': total,
            'bairro': bairro,
            'metragem': metragem,
            'quartos': quartos,
            'banheiros': banheiros,
            'vagas': vagas,
            'andar': andar,
            'aceita_pet': aceita_pet,
            'sem_mobilia': sem_mobilia,
            'latitude': latitude,
            'longitude': longitude
        }
    except Exception as e:
        print(f"Erro ao raspar dados da URL {url}: {e}")
        return None

# Carregar o CSV com as URLs
df = pd.read_csv('urls.csv')

# Lista para armazenar os dados raspados
data_list = []

# Iterar sobre as URLs e raspar os dados
for index, row in df.iterrows():
    url = row['urls']
    print(f"Raspando dados da URL: {url}")
    data = scrape_data(url)
    if data:
        data_list.append(data)

# Criar um DataFrame com os dados raspados
df_scraped = pd.DataFrame(data_list)

# Salvar os dados raspados em um novo arquivo CSV
df_scraped.to_csv('imoveis_detalhados.csv', index=False)

# Finalizar o driver do Selenium
driver.quit()


Raspando dados da URL: https://www.quintoandar.com.br/imovel/892992336/alugar/casa-2-quartos-setor-habitacional-jardim-botanico-lago-sul-brasilia?from_route=%22search_results%22&house_tags=exclusivity&search_id=%225fb08272-6d0a-49f6-8bff-97d4dca5dcb9%22&search_rank=%7B%22sortMode%22%3A%22relevance%22%2C%22searchMode%22%3A%22list%22%2C%22resultsOrigin%22%3A%22search%22%2C%22rank%22%3A0%2C%22personalization%22%3Atrue%7D
Erro ao raspar dados da URL https://www.quintoandar.com.br/imovel/892992336/alugar/casa-2-quartos-setor-habitacional-jardim-botanico-lago-sul-brasilia?from_route=%22search_results%22&house_tags=exclusivity&search_id=%225fb08272-6d0a-49f6-8bff-97d4dca5dcb9%22&search_rank=%7B%22sortMode%22%3A%22relevance%22%2C%22searchMode%22%3A%22list%22%2C%22resultsOrigin%22%3A%22search%22%2C%22rank%22%3A0%2C%22personalization%22%3Atrue%7D: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//span[text()='Aluguel']/following::p"}
  (Session info: chrome=128.

KeyboardInterrupt: 

In [14]:
# Criar um DataFrame com os dados raspados
df_scraped = pd.DataFrame(data_list)

# Salvar os dados raspados em um novo arquivo CSV
df_scraped.to_csv('imoveis_detalhados.csv', index=False)

In [6]:
%pip install webdriver-manager


[notice] A new release of pip is available: 23.3.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
